In [ ]:
!pip install numpy pandas matplotlib tensorflow scikit-learn seaborn

import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

def load_cifar10(data_dir):
    # Training data
    x_train_list, y_train_list = [], []
    for i in range(1, 6):
        batch = unpickle(os.path.join(data_dir, f'data_batch_{i}'))
        x_train_list.append(batch[b'data'])
        y_train_list.append(batch[b'labels'])
    x_train = np.concatenate(x_train_list)
    y_train = np.concatenate(y_train_list)

    # Test data
    test_batch = unpickle(os.path.join(data_dir, 'test_batch'))
    x_test = test_batch[b'data']
    y_test = np.array(test_batch[b'labels'])

    # Reshape to (num_samples, 32, 32, 3)
    x_train = x_train.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    x_test = x_test.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    return x_train, y_train, x_test, y_test

data_dir = '/content/drive/My Drive/IE4483 Mini Project/cifar-10-batches-py'
x_train, y_train, x_test, y_test = load_cifar10(data_dir)
print("Loaded data:", x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Loaded data: (50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


In [ ]:
from tensorflow.keras.utils import to_categorical

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode labels (CIFAR-10 has 10 classes)
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

IMG_HEIGHT, IMG_WIDTH = 32, 32

In [ ]:
custom_model = models.Sequential([
    # Input Layer
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    # Block 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),

    # Block 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),

    # Block 3
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),

    # Fully Connected Layers
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),

    # Output Layer (changed from 2 to 10)
    layers.Dense(num_classes, activation='softmax')
])

custom_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,148,362 (4.38 MB)

 Trainable params: 1,147,914 (4.38 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
import tensorflow as tf

custom_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
checkpoint = ModelCheckpoint(
    "best_cifar10_model.h5",
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-5,
    verbose=1
)

In [ ]:
history = custom_model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=50,
    batch_size=64,
    callbacks=[checkpoint, reduce_lr],
    verbose=1
)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - accuracy: 0.3287 - loss: 2.0442
Epoch 1: val_accuracy improved from -inf to 0.18360, saving model to best_cifar10_model.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 209s 260ms/step - accuracy: 0.3288 - loss: 2.0437 - val_accuracy: 0.1836 - val_loss: 2.7344 - learning_rate: 0.0010
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - accuracy: 0.5023 - loss: 1.3758
Epoch 2: val_accuracy did not improve from 0.18360
782/782 ━━━━━━━━━━━━━━━━━━━━ 196s 250ms/step - accuracy: 0.5023 - loss: 1.3757 - val_accuracy: 0.1008 - val_loss: 10.4656 - learning_rate: 0.0010
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.5659 - loss: 1.2234
Epoch 3: val_accuracy improved from 0.18360 to 0.26950, saving model to best_cifar10_model.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 185s 236ms/step - accuracy: 0.5659 - loss: 1.2233 - val_accuracy: 0.2695 - val_loss: 2.4019 - learning_rate: 0.0010
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.6081 - loss: 1.1093
Epoch 4: val_accuracy did not improve from 0.26950
782/782 ━━━━━━━━━━━━━━━━━━━━ 182s 233ms/step - accuracy: 0.6081 - loss: 1.1093 - val_accuracy: 0.1779 - val_loss: 4.5748 - learning_rate: 0.0010
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 0.6364 - loss: 1.0253
Epoch 5: val_accuracy improved from 0.26950 to 0.39960, saving model to best_cifar10_model.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 210s 243ms/step - accuracy: 0.6364 - loss: 1.0253 - val_accuracy: 0.3996 - val_loss: 1.6988 - learning_rate: 0.0010
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.6607 - loss: 0.9671
Epoch 6: val_accuracy did not improve from 0.39960
782/782 ━━━━━━━━━━━━━━━━━━━━ 190s 243ms/step - accuracy: 0.6607 - loss: 0.9671 - val_accuracy: 0.1077 - val_loss: 8.8931 - learning_rate: 0.0010
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - accuracy: 0.6825 - loss: 0.8990
Epoch 7: val_accuracy did not improve from 0.39960
782/782 ━━━━━━━━━━━━━━━━━━━━ 184s 236ms/step - accuracy: 0.6825 - loss: 0.8990 - val_accuracy: 0.1017 - val_loss: 9.6732 - learning_rate: 0.0010
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step - accuracy: 0.7024 - loss: 0.8577
Epoch 8: val_accuracy did not improve from 0.39960
782/782 ━━━━━━━━━━━━━━━━━━━━ 211s 248ms/step - accuracy: 0.7024 - loss: 0.8577 - val_accuracy: 0.1057 - val_loss: 6.7409 - learning_rate: 0.0010
Epo


Epoch 25: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
782/782 ━━━━━━━━━━━━━━━━━━━━ 201s 222ms/step - accuracy: 0.8404 - loss: 0.4515 - val_accuracy: 0.4999 - val_loss: 1.9629 - learning_rate: 1.2500e-04
Epoch 26/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - accuracy: 0.8458 - loss: 0.4369
Epoch 26: val_accuracy improved from 0.49990 to 0.51860, saving model to best_cifar10_model.h5


...

782/782 ━━━━━━━━━━━━━━━━━━━━ 177s 227ms/step - accuracy: 0.8569 - loss: 0.4050 - val_accuracy: 0.7311 - val_loss: 0.8334 - learning_rate: 3.1250e-05
Epoch 36/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.8585 - loss: 0.4066
Epoch 36: val_accuracy improved from 0.73110 to 0.75730, saving model to best_cifar10_model.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 202s 226ms/step - accuracy: 0.8585 - loss: 0.4066 - val_accuracy: 0.7573 - val_loss: 0.8036 - learning_rate: 3.1250e-05
Epoch 37/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.8563 - loss: 0.4031
Epoch 37: val_accuracy did not improve from 0.75730
782/782 ━━━━━━━━━━━━━━━━━━━━ 200s 224ms/step - accuracy: 0.8563 - loss: 0.4031 - val_accuracy: 0.5688 - val_loss: 1.5615 - learning_rate: 3.1250e-05
Epoch 38/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.8575 - loss: 0.3970
Epoch 38: val_accuracy did not improve from 0.75730
782/782 ━━━━━━━━━━━━━━━━━━━━ 175s 224ms/step - accuracy: 0.8575 - loss: 0.3970 - val_accuracy: 0.6447 - val_loss: 1.2715 - learning_rate: 3.1250e-05
Epoch 39/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.8609 - loss: 0.3915
Epoch 39: val_accuracy did not improve from 0.75730
782/782 ━━━━━━━━━━━━━━━━━━━━ 178s 228ms/step - accuracy: 0.8609 - loss: 0.3915 - val_accuracy: 0.6009 - val_loss: 1.4793 - learnin

782/782 ━━━━━━━━━━━━━━━━━━━━ 176s 225ms/step - accuracy: 0.8608 - loss: 0.3974 - val_accuracy: 0.7775 - val_loss: 0.7153 - learning_rate: 1.5625e-05
Epoch 43/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.8610 - loss: 0.3919
Epoch 43: val_accuracy did not improve from 0.77750
782/782 ━━━━━━━━━━━━━━━━━━━━ 175s 224ms/step - accuracy: 0.8610 - loss: 0.3919 - val_accuracy: 0.7567 - val_loss: 0.7845 - learning_rate: 1.5625e-05
Epoch 44/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.8612 - loss: 0.3930
Epoch 44: val_accuracy did not improve from 0.77750
782/782 ━━━━━━━━━━━━━━━━━━━━ 202s 224ms/step - accuracy: 0.8612 - loss: 0.3930 - val_accuracy: 0.7140 - val_loss: 0.9395 - learning_rate: 1.5625e-05
Epoch 45/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step - accuracy: 0.8673 - loss: 0.3736
Epoch 45: val_accuracy improved from 0.77750 to 0.77950, saving model to best_cifar10_model.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 179s 229ms/step - accuracy: 0.8673 - loss: 0.3736 - val_accuracy: 0.7795 - val_loss: 0.7069 - learning_rate: 1.5625e-05
Epoch 46/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.8655 - loss: 0.3834
Epoch 46: val_accuracy improved from 0.77950 to 0.78200, saving model to best_cifar10_model.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 182s 232ms/step - accuracy: 0.8655 - loss: 0.3834 - val_accuracy: 0.7820 - val_loss: 0.7100 - learning_rate: 1.5625e-05
Epoch 47/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.8609 - loss: 0.3915
Epoch 47: val_accuracy did not improve from 0.78200
782/782 ━━━━━━━━━━━━━━━━━━━━ 199s 229ms/step - accuracy: 0.8609 - loss: 0.3915 - val_accuracy: 0.7548 - val_loss: 0.8008 - learning_rate: 1.5625e-05
Epoch 48/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.8624 - loss: 0.3854
Epoch 48: val_accuracy did not improve from 0.78200
782/782 ━━━━━━━━━━━━━━━━━━━━ 203s 230ms/step - accuracy: 0.8624 - loss: 0.3854 - val_accuracy: 0.7679 - val_loss: 0.7369 - learning_rate: 1.5625e-05
Epoch 49/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.8631 - loss: 0.3854
Epoch 49: val_accuracy did not improve from 0.78200
782/782 ━━━━━━━━━━━━━━━━━━━━ 201s 229ms/step - accuracy: 0.8631 - loss: 0.3854 - val_accuracy: 0.7451 - val_loss: 0.8569 - learnin

In [ ]:

test_loss, test_acc = custom_model.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy: {test_acc:.4f}")

313/313 - 14s - 46ms/step - accuracy: 0.6651 - loss: 1.1855
Test Accuracy: 0.6651
